In [1]:
import random
import numpy as np
from scipy import sparse
import pandas as pd
from tools.luDecomposition import lu_decomposition
from tools.inverseMatrix import inverse_matrix
from tools.linearEquationsSystemSolve import linear_equations_system_solve
from tools.seidel import seidel
from tools.conditionalNumber import get_conditional_number

In [2]:
def generate_gilbert_matrix(k: int):
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = 1 / (i + j + 1)
    return matrix

In [3]:
def generate_diagonal_matrix(k):
    values = [0, -1, -2, -3, -4, -5, -6]
    noise = 10 ** (-k)
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = random.choice(values)
    for i in range(k):
        matrix[i][i] = -(sum(matrix[i]) - matrix[i][i]) + noise
    return matrix

In [4]:
def solutions(a: sparse.csr_matrix, b: np.array, short_output: bool = False):
    if not short_output:
        print("a =", a.todense())
        print("b =", b)
        print("Conditional number = ", get_conditional_number(a))
        print(">--------------------------------------------------------------<\n")

    answer_lu, iteration_count = linear_equations_system_solve(a, sparse.csr_matrix(b))
    print("Solution of system A with B vector (lu method)")
    if not short_output:
        print(answer_lu.transpose().toarray())

    print("Iteration number")
    print(iteration_count)
    print(">--------------------------------------------------------------<\n")

    answer_sei, iteration_count = seidel(a, b, 0.0001)
    print("Solution of system A with B vector (Seidel method)")
    if not short_output:
        print(answer_sei)
    print("Iteration number")
    print(iteration_count)
    print(">--------------------------------------------------------------<\n")

    print("Error for lu")
    print(np.linalg.norm(np.subtract(b, a.dot(answer_lu.transpose().toarray()[0]))))
    print(">--------------------------------------------------------------<\n")

    print("Error for Seidel")
    print(np.linalg.norm(np.subtract(b, a.dot(answer_sei))))
    print(">==============================================================<\n\n")

In [5]:
print("Eye matrix 3x3")
solutions(sparse.csr_matrix(np.eye(3, 3)),
          np.array([1., 2., 3.]))

Random matrix 3x3
a = [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
b = [1. 2. 3.]
Conditional number =  2.9999999999999996
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[1. 2. 3.]]
Iteration number
35
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[1. 2. 3.]
Iteration number
18
>--------------------------------------------------------------<

Error for lu
0.0
>--------------------------------------------------------------<

Error for Seidel
0.0
>==============================================================<




In [6]:
print("Random matrix 4x4")
solutions(sparse.csr_matrix([[20., 0., 5., 4.],
                             [-7., 3., -1., 9],
                             [10., 2., -4., 2],
                             [2., 9., 11., 55.]])
              ,np.array([1., 2., 3., 4.]))


Random matrix 4x4
a = [[20.  0.  5.  4.]
 [-7.  3. -1.  9.]
 [10.  2. -4.  2.]
 [ 2.  9. 11. 55.]]
b = [1. 2. 3. 4.]
Conditional number =  86.60617988340239
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[ 7.69230769e-02  1.25000000e+00  7.93016446e-17 -1.34615385e-01]]
Iteration number
69
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[ 7.69279727e-02  1.24985097e+00 -3.87149027e-05 -1.34583433e-01]
Iteration number
352
>--------------------------------------------------------------<

Error for lu
2.7012892057857038e-15
>--------------------------------------------------------------<

Error for Seidel
0.00016125372834932436
>==============================================================<




In [7]:
print("Diagonal matrix 5x5 #1")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 5x5 #2")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 5x5 #3")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 10x10")
n = 10
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Diagonal matrix 50x50")
n = 50
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Diagonal matrix 100x100")
n = 100
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

Diagonal matrix 5x5 #1
a = [[ 7.00001 -3.      -4.       0.       0.     ]
 [-4.       8.00001 -2.       0.      -2.     ]
 [-2.       0.       7.00001 -5.       0.     ]
 [-3.      -2.       0.       6.00001 -1.     ]
 [-5.      -3.      -6.      -2.      16.00001]]
b = [-10.99999  -3.99998  -0.99997  12.00004  43.00005]
Conditional number =  2731117.393505214
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[1. 2. 3. 4. 5.]]
Iteration number
119
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[-2.35923582 -1.3592392  -0.35921472  0.64076918  1.64077418]
Iteration number
250
>--------------------------------------------------------------<

Error for lu
1.4862067603866e-14
>--------------------------------------------------------------<

Error for Seidel
0.0001863205577993837
>==============================================================<


Diagonal mat

In [8]:
print("Gilbert matrix 5x5")
n = 5
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Gilbert matrix 10x10")
n = 10
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Gilbert matrix 50x50")
n = 50
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

Gilbert matrix 5x5
a = [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
b = [5.         3.55       2.81428571 2.34642857 2.01746032]
Conditional number =  480849.1169944144
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[1. 2. 3. 4. 5.]]
Iteration number
119
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[0.99999625 2.02639721 2.82444736 4.32423602 4.82135593]
Iteration number
34725
>--------------------------------------------------------------<

Error for lu
0.0
>--------------------------------------------------------------<

Error for Seidel
2.205593874610787e-05
>==============================================

In [28]:
def build_table(matrix, eps):

    seidel_res = []
    lu_res = []
    obusl = []
    sizes = [5, 10, 50, 100]
    precisions = [eps for i in sizes]

    for n in sizes:
        a = sparse.csr_matrix(matrix(n))
        ans = np.array([i for i in range(1, n + 1)])
        b = a.dot(ans)
        obusl.append(get_conditional_number(a.copy()))
        answer_lu, it1 = linear_equations_system_solve(a.copy(), sparse.csr_matrix(b.copy()))
        answer_sei, it2 = seidel(a, b, eps)
        lu_res.append(it1)
        seidel_res.append(it2)

    data = {"size n": sizes,
            "epsilon": precisions,
            "Cond(A)": obusl,
            "LU iterations": lu_res,
            "Seidel iterations": seidel_res}

    df = pd.DataFrame(data)

    return df

In [29]:
build_table(generate_diagonal_matrix, 0.01)

,size n,epsilon,Cond(A),LU iterations,Seidel iterations
0,5,0.01,3.129004e+06,119,100
1,10,0.01,9.223961e+11,714,500
2,50,0.01,1.499922e+17,67574,15000
3,100,0.01,1.242334e+17,520149,60000


In [30]:
build_table(generate_gilbert_matrix, 0.01)

,size n,epsilon,Cond(A),LU iterations,Seidel iterations
0,5,0.01,4.808491e+05,119,1375
1,10,0.01,1.633163e+13,714,16300
2,50,0.01,1.765061e+19,67574,100000
3,100,0.01,2.007829e+19,520149,100000


In [49]:
def build_table2(eps):

    seidel_res = []
    lu_res = []
    obusl = []
    sizes = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    precisions = [eps for i in sizes]

    for n in sizes:
        a = sparse.csr_matrix(np.random.uniform(low=1, high=100, size=(n, n)))
        ans = np.array([i for i in range(1, n + 1)])
        b = a.dot(ans)
        obusl.append(get_conditional_number(a.copy()))
        answer_lu, it1 = linear_equations_system_solve(a.copy(), sparse.csr_matrix(b.copy()))
        answer_sei, it2 = seidel(a, b, eps)
        lu_res.append(it1)
        seidel_res.append(it2)

    data = {"size n": sizes,
            "epsilon": precisions,
            "Cond(A)": obusl,
            "LU iterations": lu_res,
            "Seidel iterations": seidel_res}

    df = pd.DataFrame(data)

    return df

In [50]:
build_table2(0.01)

D:\ITMO_AppliedMath\Lab3\tools\seidel.py:28: RuntimeWarning: overflow encountered in double_scalars
  s = s + A[i, j] * x[j]
D:\ITMO_AppliedMath\Lab3\tools\seidel.py:28: RuntimeWarning: invalid value encountered in double_scalars
  s = s + A[i, j] * x[j]


,size n,epsilon,Cond(A),LU iterations,Seidel iterations
0,5,0.01,56.926044,119,23650
1,10,0.01,48.103055,714,15100
2,20,0.01,4538.432128,4829,53600
3,30,0.01,304.302476,15344,50400
4,40,0.01,3797.965538,35259,41600
5,50,0.01,694.598459,67574,100000
6,60,0.01,2001.536347,115289,100800
7,70,0.01,1506.864974,181404,102900
8,80,0.01,1398.620929,268919,102400
9,90,0.01,7886.013817,380834,105300
